In [1]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import make_pipeline

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('../datasets/BankChurners_change.csv', encoding='CP949')
data = data.iloc[:,2:23]
np.random.seed(42)
random.seed(42)

In [12]:
object_columns = data.select_dtypes('object').columns
for i in object_columns:
    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']

category : [0 1]
classes : ['F' 'M']

category : [0 1 2 3 4 5 6]
classes : ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated' 'Unknown']

category : [0 1 2 3]
classes : ['Divorced' 'Married' 'Single' 'Unknown']

category : [0 1 2 3 4 5]
classes : ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K'
 'Unknown']

category : [0 1 2 3]
classes : ['Blue' 'Gold' 'Platinum' 'Silver']

category : [0 1 2 3 4]
classes : ['20대' '30대' '40대' '50대' '60대 이상']



## Under Sampling

In [13]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i == OneSidedSelection:
        sampling = i(random_state=42)
    else:
        sampling = i()
        
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    xgb = XGBClassifier(random_state=42)
    xgb.fit(x_train_ss, y_train)
    pred = xgb.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("XGB Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

XGB Model's score by Under sampling


,f1_score,recall,precision,accuracy
TomekLinks,0.979173,0.982343,0.976023,0.964956
NeighbourhoodCleaningRule,0.978158,0.975280,0.981054,0.963475
RepeatedEditedNearestNeighbours,0.978015,0.968805,0.987403,0.963475
OneSidedSelection,0.977999,0.981165,0.974854,0.962981
EditedNearestNeighbours,0.977212,0.971748,0.982738,0.961994
AllKNN,0.976551,0.968217,0.985030,0.961007
NearMiss,0.928727,0.878164,0.985469,0.886969


## Over Sampling

In [14]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy, random_state=42)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        xgb = XGBClassifier(random_state=42)
        xgb.fit(x_train_ss, y_train)
        pred = xgb.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("XGB Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

XGB Model's score by Over sampling


,f1_score,recall,precision,accuracy
SMOTE(0.3),0.974084,0.962331,0.986128,0.957058
ADASYN(0.3),0.969970,0.950559,0.990190,0.950642
BorderlineSMOTE(0.3),0.966787,0.942319,0.992560,0.945706
SMOTE(0.4),0.949538,0.908181,0.994842,0.919052
SMOTE(0.6),0.940484,0.892878,0.993451,0.905232
SMOTE(0.5),0.938979,0.887581,0.996695,0.903258
ADASYN(0.4),0.938471,0.888758,0.994075,0.902270
BorderlineSMOTE(0.4),0.933083,0.878164,0.995330,0.894373
BorderlineSMOTE(0.5),0.917355,0.849323,0.997236,0.871668
SMOTE(0.7),0.912225,0.841083,0.996513,0.864265


## Combine Sampling

In [15]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i(random_state=42)
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    xgb = XGBClassifier(random_state=42)
    xgb.fit(x_train_ss, y_train)
    pred = xgb.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("XGB Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

XGB Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTEENN,0.90880,0.835786,0.995792,0.859329
SMOTETomek,0.88071,0.788699,0.997024,0.820829


## Pipeline을 통한 Combine Sampling

In [16]:
strategy_ = [0.3, 0.4, 0.5]
columns = ['f1_score', 'recall', 'precision', 'accuracy']

df = pd.DataFrame(columns=columns)

under_sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]
over_sampling = [SMOTE, BorderlineSMOTE, ADASYN]

xgb = XGBClassifier()

for over in over_sampling:

    for under in under_sampling:

        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        for strategy in strategy_:
            
            if under == OneSidedSelection:
                under_ = under(random_state=42)
            else:
                under_ = under()

            over_ = over(sampling_strategy=strategy, random_state=42)

            ss = StandardScaler()
            x_train_ss = ss.fit_transform(x_train)
            x_test_ss = ss.fit_transform(x_test)

            model = make_pipeline(over_, under_, xgb)   
            model.fit(x_train_ss, y_train)
            pred = model.predict(x_test_ss)

            acc = accuracy_score(y_test, pred)
            recall = recall_score(y_test, pred)
            precision = precision_score(y_test, pred)
            f1 = f1_score(y_test, pred)

            df.loc[over.__name__+under.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("XGB Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

XGB Model's score by sampling


,f1_score,recall,precision,accuracy
SMOTEEditedNearestNeighbours(0.3),0.971856,0.955268,0.989031,0.953603
SMOTENeighbourhoodCleaningRule(0.3),0.971240,0.954091,0.989018,0.952616
SMOTEOneSidedSelection(0.3),0.970897,0.952325,0.990208,0.952122
SMOTEAllKNN(0.3),0.970006,0.951736,0.988991,0.950642
SMOTETomekLinks(0.3),0.969115,0.951148,0.987775,0.949161
ADASYNOneSidedSelection(0.3),0.968967,0.946439,0.992593,0.949161
SMOTERepeatedEditedNearestNeighbours(0.3),0.967800,0.946439,0.990148,0.947187
BorderlineSMOTETomekLinks(0.3),0.967450,0.944673,0.991353,0.946693
BorderlineSMOTENeighbourhoodCleaningRule(0.3),0.966847,0.944085,0.990735,0.945706
BorderlineSMOTEOneSidedSelection(0.3),0.965204,0.938788,0.993151,0.943238


## Under, Over, Combine Sampling과 Pipeline을 통한 Combine Sampling 전체 결과

In [17]:
sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i in [OneSidedSelection, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]:
        sampling = i(random_state=42)
    else:
        sampling = i()

    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    xgb = XGBClassifier(random_state=42)
    xgb.fit(x_train_ss, y_train)
    pred = xgb.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[i.__name__] = [f1, recall, precision, acc]

print("XGB Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

XGB Model's score by sampling


,f1_score,recall,precision,accuracy
TomekLinks,0.979173,0.982343,0.976023,0.964956
NeighbourhoodCleaningRule,0.978158,0.975280,0.981054,0.963475
RepeatedEditedNearestNeighbours,0.978015,0.968805,0.987403,0.963475
OneSidedSelection,0.977999,0.981165,0.974854,0.962981
EditedNearestNeighbours,0.977212,0.971748,0.982738,0.961994
AllKNN,0.976551,0.968217,0.985030,0.961007
SMOTEEditedNearestNeighbours(0.3),0.971856,0.955268,0.989031,0.953603
SMOTENeighbourhoodCleaningRule(0.3),0.971240,0.954091,0.989018,0.952616
SMOTEOneSidedSelection(0.3),0.970897,0.952325,0.990208,0.952122
SMOTEAllKNN(0.3),0.970006,0.951736,0.988991,0.950642


## No Sampling

In [18]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
NO_Sampling = pd.DataFrame(columns=columns)

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.fit_transform(x_test)


xgb = XGBClassifier(random_state=42)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

NO_Sampling.loc['No Sampling'] = [f1, recall, precision, acc]

print("XGB Model's score by sampling")
NO_Sampling.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
NO_Sampling.head(10)

XGB Model's score by sampling


,f1_score,recall,precision,accuracy
No Sampling,0.978861,0.981165,0.976567,0.964462
